In [1]:
#import required libraries
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_DETERMINISTIC_OPS'] = '1'
import cv2
import numpy as np
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import lightgbm
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from tensorflow.keras.applications import VGG16, ResNet50, EfficientNetB0
from sklearn.metrics import classification_report, roc_auc_score, matthews_corrcoef, precision_score, f1_score, recall_score
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, SimpleRNN,Dropout
from tensorflow.keras.utils import to_categorical
import pandas as pd
import time
import psutil
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
import keras

tf.keras.utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)
tf.config.experimental.enable_tensor_float_32_execution(True)
tf.config.threading.set_inter_op_parallelism_threads(1)
tf.config.threading.set_intra_op_parallelism_threads(1)

2024-03-13 16:06:41.951225: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-13 16:06:41.951303: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-13 16:06:41.954076: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-13 16:06:41.973401: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-13 16:06:44.472132: W ten

In [2]:
#!pip install tensorflow-tensorrt


In [3]:
#!pip install opencv-python
#!pip install imutils
#!pip install scikit-learn
#!pip install xgboost


In [4]:
#Class to load the dataset images from device

class SimpleDatasetLoader:
    # Method: Constructor
    def __init__(self, preprocessors=None):
        """
        :param preprocessors: List of image preprocessors
        """
        self.preprocessors = preprocessors

        if self.preprocessors is None:
            self.preprocessors = []

    # Method: Used to load a list of images for pre-processing
    def load(self, image_paths, verbose=-1):
        """
        :param image_paths: List of image paths
        :param verbose: Parameter for printing information to console
        :return: Tuple of data and labels
        """
        data, labels = [], []

        for i, image_path in enumerate(image_paths):
            image = cv2.imread(image_path)
            label = image_path.split(os.path.sep)[-2]

            if self.preprocessors is not None:
                for p in self.preprocessors:
                    image = p.preprocess(image)

            data.append(image)
            labels.append(label)

            if verbose > 0 and i > 0 and (i+1) % verbose == 0:
                print('[INFO]: Processed {}/{}'.format(i+1, len(image_paths)))

        return (np.array(data), np.array(labels))

In [5]:
#Class Preprocessror 
class SimplePreprocessor:
    # Method: Constructor
    def __init__(self, width, height, interpolation=cv2.INTER_AREA):
        """
        :param width: Image width
        :param height: Image height
        :param interpolation: Interpolation algorithm
        """
        self.width = width
        self.height = height
        self.interpolation = interpolation

    # Method: Used to resize the image to a fixed size (ignoring the aspect ratio)
    def preprocess(self, image):
        """
        :param image: Image
        :return: Re-sized image
        """
        return cv2.resize(image, (self.width, self.height), interpolation=self.interpolation)

In [6]:
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from __main__ import SimplePreprocessor
from __main__ import SimpleDatasetLoader

In [7]:
# Function to load and preprocess data using SimpleDatasetLoader
def load_and_preprocess_data(image_paths, target_size):
    sp = SimplePreprocessor(target_size[0], target_size[1])
    sdl = SimpleDatasetLoader(preprocessors=[sp])
    data, labels = sdl.load(image_paths, verbose=1000)

    print(data.shape)
    print(labels.shape)

    # Convert labels to one-hot encoding
    le = LabelEncoder()
    labels = le.fit_transform(labels)
    #labels = to_categorical(labels)

    return data, labels

# Get list of image paths
image_paths = list(paths.list_images("../BreaKHis_v1"))

# Define target size for images
target_size = (224, 224)  # Change this to your desired size

# Load and preprocess data
print('[INFO]: Images loading....')
data, labels = load_and_preprocess_data(image_paths, target_size)

[INFO]: Images loading....
[INFO]: Processed 1000/7909
[INFO]: Processed 2000/7909
[INFO]: Processed 3000/7909
[INFO]: Processed 4000/7909
[INFO]: Processed 5000/7909
[INFO]: Processed 6000/7909
[INFO]: Processed 7000/7909
(7909, 224, 224, 3)
(7909,)


In [8]:
import gc
import os

# Initialize a DataFrame to store results
results_df = pd.DataFrame(columns=['Model', 'Average Accuracy', 'Average Sensitivity', 'Average Specificity',
                                   'Average AUC-ROC', 'Average MCC', 'Average Precision', 'Average F1 Score',
                                   'Memory Used (MB)', 'Time (s)'])

# Function to evaluate a model
def evaluate_model(model, name, data, labels):
    if name == "VGG16" or name == "EfficientNetB0" or name == "ResNet50" :
        labels = to_categorical(labels)

    else:
        data = data.reshape(data.shape[0],data.shape[1] * data.shape[2] * data.shape[3] )
    accuracy_list = []
    sensitivity_list = []
    specificity_list = []
    auc_roc_list = []
    mcc_list = []
    precision_list = []
    f1_list = []
    time_start = time.time()
    memory_start = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2

    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
    if name == "VGG16" or name == "EfficientNetB0" or name == "ResNet50":


        with tf.device('/GPU:0'):
            tf.random.set_seed(0)
            for fold, (train_index, test_index) in enumerate(skf.split(data, labels.argmax(axis=1))):
                print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

                X_train, X_test = data[train_index], data[test_index]
                y_train, y_test = labels[train_index], labels[test_index]


                # Adding early stopping to prevent overfitting
                early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
                keras.utils.set_random_seed(0)
                np.random.seed(0)
                tf.random.set_seed(0)
                #tf.config.run_functions_eagerly(True)
                tf.data.experimental.enable_debug_mode()
                # Train the model
                # Example of using tf.data.Dataset
                train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
                train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(64)

                model.fit(train_dataset, epochs=20, callbacks=[early_stopping])

                # Evaluate the model
                predictions = model.predict(X_test)
                y_pred = np.argmax(predictions, axis=1)
                y_test_encoded = np.argmax(y_test, axis=1)

                
                # Calculate evaluation metrics for the current fold
                accuracy = np.mean(y_pred == y_test_encoded)
                sensitivity = recall_score(y_test_encoded, y_pred, pos_label=1)
                specificity = recall_score(y_test_encoded, y_pred, pos_label=0)
                auc_roc = roc_auc_score(y_test_encoded, y_pred)
                mcc = matthews_corrcoef(y_test_encoded, y_pred)
                precision = precision_score(y_test_encoded, y_pred, pos_label=1)
                f1 = f1_score(y_test_encoded, y_pred, pos_label=1)
                # Append metrics to lists
                accuracy_list.append(accuracy)
                sensitivity_list.append(sensitivity)
                specificity_list.append(specificity)
                auc_roc_list.append(auc_roc)
                mcc_list.append(mcc)
                precision_list.append(precision)
                f1_list.append(f1)

                # Print metrics for the current fold
                print(f"Accuracy: {accuracy}")
                print(f"Sensitivity: {sensitivity}")
                print(f"Specificity: {specificity}")
                print(f"AUC-ROC: {auc_roc}")
                print(f"MCC: {mcc}")
                print(f"Precision: {precision}")
                print(f"F1 Score: {f1}")
                gc.collect()

            # Calculate average metrics
            average_accuracy = np.mean(accuracy_list)
            average_sensitivity = np.mean(sensitivity_list)
            average_specificity = np.mean(specificity_list)
            average_auc_roc = np.mean(auc_roc_list)
            average_mcc = np.mean(mcc_list)
            average_precision = np.mean(precision_list)
            average_f1 = np.mean(f1_list)

            time_end = time.time()
            memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
            del X_train, X_test, y_train, y_test
            gc.collect()
            tf.keras.backend.clear_session()
    else:
        for fold, (train_index, test_index) in enumerate(skf.split(data, labels)):
            print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

            X_train, X_test = data[train_index], data[test_index]
            y_train, y_test = labels[train_index], labels[test_index]

            # Train the model
            model.fit(X_train, y_train)

            # Evaluate the model
            predictions = model.predict(X_test)
            y_pred = predictions
            y_test_encoded = y_test

            # Calculate evaluation metrics for the current fold
            accuracy = np.mean(y_pred == y_test_encoded)
            sensitivity = recall_score(y_test, y_pred, pos_label=1)
            specificity = recall_score(y_test, y_pred, pos_label=0)
            auc_roc = roc_auc_score(y_test, predictions)
            mcc = matthews_corrcoef(y_test, y_pred)
            precision = precision_score(y_test, y_pred, pos_label=1)
            f1 = f1_score(y_test, y_pred, pos_label=1)
            # Append metrics to lists
            accuracy_list.append(accuracy)
            sensitivity_list.append(sensitivity)
            specificity_list.append(specificity)
            auc_roc_list.append(auc_roc)
            mcc_list.append(mcc)
            precision_list.append(precision)
            f1_list.append(f1)

            # Print metrics for the current fold
            print(f"Accuracy: {accuracy}")
            print(f"Sensitivity: {sensitivity}")
            print(f"Specificity: {specificity}")
            print(f"AUC-ROC: {auc_roc}")
            print(f"MCC: {mcc}")
            print(f"Precision: {precision}")
            print(f"F1 Score: {f1}")
            gc.collect()

        # Calculate average metrics
        average_accuracy = np.mean(accuracy_list)
        average_sensitivity = np.mean(sensitivity_list)
        average_specificity = np.mean(specificity_list)
        average_auc_roc = np.mean(auc_roc_list)
        average_mcc = np.mean(mcc_list)
        average_precision = np.mean(precision_list)
        average_f1 = np.mean(f1_list)

        time_end = time.time()
        memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
        

    # Append results to DataFrame
    results_df.loc[len(results_df)] = [name, average_accuracy, average_sensitivity, average_specificity,
                                       average_auc_roc, average_mcc, average_precision, average_f1,
                                       memory_end - memory_start, time_end - time_start]


In [9]:
from sklearn.linear_model import LogisticRegression
# LR
lr_model = LogisticRegression(n_jobs=-1, random_state=0)
evaluate_model(lr_model, 'LR', data, labels)


[INFO] Fold 1 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.7623261694058154
Sensitivity: 0.8637200736648251
Specificity: 0.5403225806451613
AUC-ROC: 0.7020213271549931
MCC: 0.42578173354089105
Precision: 0.8044596912521441
F1 Score: 0.83303730017762

[INFO] Fold 2 / 10 for LR
Accuracy: 0.8015170670037927
Sensitivity: 0.8766114180478821
Specificity: 0.6370967741935484
AUC-ROC: 0.7568540961207152
MCC: 0.5282539730974752
Precision: 0.8409893992932862
F1 Score: 0.8584310189359783

[INFO] Fold 3 / 10 for LR
Accuracy: 0.7850821744627055
Sensitivity: 0.8802946593001841
Specificity: 0.5766129032258065
AUC-ROC: 0.7284537812629953
MCC: 0.48149098112548216
Precision: 0.8198970840480274
F1 Score: 0.8490230905861456

[INFO] Fold 4 / 10 for LR
Accuracy: 0.7547408343868521
Sensitivity: 0.8416206261510129
Specificity: 0.5645161290322581
AUC-ROC: 0.7030683775916354
MCC: 0.4170802319928461
Precision: 0.8088495575221238
F1 Score: 0.8249097472924188

[INFO] Fold 5 / 10 for LR
Accuracy: 0.7825537294563844
Sensitivity: 0.8821362799263351
Specificity: 0.

In [10]:



#NB
nb_model = GaussianNB()
evaluate_model(nb_model, 'NB', data, labels)


[INFO] Fold 1 / 10 for NB
Accuracy: 0.7218710493046776
Sensitivity: 0.6850828729281768
Specificity: 0.8024193548387096
AUC-ROC: 0.7437511138834433
MCC: 0.4532736258005371
Precision: 0.8836104513064132
F1 Score: 0.7717842323651453

[INFO] Fold 2 / 10 for NB
Accuracy: 0.7193426042983565
Sensitivity: 0.6703499079189686
Specificity: 0.8266129032258065
AUC-ROC: 0.7484814055723876
MCC: 0.46130358435968266
Precision: 0.8943488943488943
F1 Score: 0.7663157894736842

[INFO] Fold 3 / 10 for NB
Accuracy: 0.7041719342604298
Sensitivity: 0.6740331491712708
Specificity: 0.7701612903225806
AUC-ROC: 0.7220972197469258
MCC: 0.4131471677230494
Precision: 0.8652482269503546
F1 Score: 0.7577639751552797

[INFO] Fold 4 / 10 for NB
Accuracy: 0.6814159292035398
Sensitivity: 0.6500920810313076
Specificity: 0.75
AUC-ROC: 0.7000460405156538
MCC: 0.37167875688515256
Precision: 0.8506024096385543
F1 Score: 0.7369519832985386

[INFO] Fold 5 / 10 for NB
Accuracy: 0.7079646017699115
Sensitivity: 0.6850828729281768


In [11]:

# KNN
knn_model = KNeighborsClassifier()
evaluate_model(knn_model, 'KNN', data, labels)




[INFO] Fold 1 / 10 for KNN
Accuracy: 0.8192161820480405
Sensitivity: 0.8968692449355433
Specificity: 0.6491935483870968
AUC-ROC: 0.77303139666132
MCC: 0.5677831872924421
Precision: 0.8484320557491289
F1 Score: 0.8719785138764549

[INFO] Fold 2 / 10 for KNN
Accuracy: 0.8255372945638433
Sensitivity: 0.9116022099447514
Specificity: 0.6370967741935484
AUC-ROC: 0.7743494920691498
MCC: 0.580027210853511
Precision: 0.8461538461538461
F1 Score: 0.8776595744680852

[INFO] Fold 3 / 10 for KNN
Accuracy: 0.8116308470290771
Sensitivity: 0.8968692449355433
Specificity: 0.625
AUC-ROC: 0.7609346224677717
MCC: 0.5474345890054283
Precision: 0.8396551724137931
F1 Score: 0.8673196794300979

[INFO] Fold 4 / 10 for KNN
Accuracy: 0.8027812895069533
Sensitivity: 0.8987108655616943
Specificity: 0.592741935483871
AUC-ROC: 0.7457264005227827
MCC: 0.5228465392821294
Precision: 0.8285229202037352
F1 Score: 0.8621908127208481

[INFO] Fold 5 / 10 for KNN
Accuracy: 0.8015170670037927
Sensitivity: 0.8839779005524862


In [12]:
# Random Forest
rf_model = RandomForestClassifier(random_state=0, n_jobs=-1)
evaluate_model(rf_model, 'Random Forest', data, labels)



[INFO] Fold 1 / 10 for Random Forest
Accuracy: 0.8457648546144121
Sensitivity: 0.9429097605893186
Specificity: 0.6330645161290323
AUC-ROC: 0.7879871383591754
MCC: 0.6277565640750644
Precision: 0.8490878938640133
F1 Score: 0.8935427574171031

[INFO] Fold 2 / 10 for Random Forest
Accuracy: 0.8596713021491783
Sensitivity: 0.9465930018416207
Specificity: 0.6693548387096774
AUC-ROC: 0.8079739202756491
MCC: 0.6630240113562247
Precision: 0.8624161073825504
F1 Score: 0.9025460930640913

[INFO] Fold 3 / 10 for Random Forest
Accuracy: 0.854614412136536
Sensitivity: 0.9355432780847146
Specificity: 0.6774193548387096
AUC-ROC: 0.8064813164617122
MCC: 0.6510630873903074
Precision: 0.8639455782312925
F1 Score: 0.8983200707338638

[INFO] Fold 4 / 10 for Random Forest
Accuracy: 0.8331226295828066
Sensitivity: 0.9337016574585635
Specificity: 0.6129032258064516
AUC-ROC: 0.7733024416325075
MCC: 0.5957467499766298
Precision: 0.8407960199004975
F1 Score: 0.8848167539267014

[INFO] Fold 5 / 10 for Random Fo

In [13]:
import lightgbm
# LGBM
lgb_model = lightgbm.LGBMClassifier(random_state=0, n_jobs = -1)
evaluate_model(lgb_model, 'LGBM', data, labels)


[INFO] Fold 1 / 10 for LGBM
[LightGBM] [Info] Number of positive: 4886, number of negative: 2232
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 33.788181 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26906684
[LightGBM] [Info] Number of data points in the train set: 7118, number of used features: 150528
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.686429 -> initscore=0.783476
[LightGBM] [Info] Start training from score 0.783476
Accuracy: 0.8533501896333755
Sensitivity: 0.9263351749539595
Specificity: 0.6935483870967742
AUC-ROC: 0.8099417810253668
MCC: 0.6492749729998661
Precision: 0.8687392055267703
F1 Score: 0.8966131907308378

[INFO] Fold 2 / 10 for LGBM
[LightGBM] [Info] Number of positive: 4886, number of negative: 2232
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 33.698845 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGB

In [14]:
# Set random seed for reproducibility
np.random.seed(0)
tf.random.set_seed(0)

# Load the VGG16 model pre-trained on ImageNet data
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers
for layer in vgg16_model.layers:
    layer.trainable = False

# Build a new model using VGG16 as a feature extractor
with tf.device('/GPU:0'):
    model_vgg16 = Sequential()
    model_vgg16.add(vgg16_model)
    model_vgg16.add(Flatten())
    model_vgg16.add(Dense(256, activation='relu', kernel_initializer='glorot_normal', bias_initializer='zeros'))
    model_vgg16.add(Dropout(0.5, seed = 0))
    model_vgg16.add(Dense(2, activation='softmax', kernel_initializer='glorot_normal', bias_initializer='zeros'))

    # Compile the model
    model_vgg16.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    evaluate_model(model_vgg16, 'VGG16', data, labels)

2024-03-13 20:42:11.552454: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 37960 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:65:00.0, compute capability: 8.0



[INFO] Fold 1 / 10 for VGG16


2024-03-13 20:42:14.869466: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Epoch 1/20


2024-03-13 20:42:20.186325: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-13 20:42:20.497461: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-13 20:42:23.710636: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fa153213e00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-13 20:42:23.710693: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-SXM4-40GB, Compute Capability 8.0
2024-03-13 20:42:23.726384: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1710362544.036346  558553 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


112/112 [==============================] - 13s 48ms/step - loss: 1.8287 - accuracy: 0.7602
Epoch 2/20
112/112 [==============================] - 5s 45ms/step - loss: 0.3529 - accuracy: 0.8484
Epoch 3/20
112/112 [==============================] - 5s 45ms/step - loss: 0.2941 - accuracy: 0.8748
Epoch 4/20
112/112 [==============================] - 5s 46ms/step - loss: 0.2578 - accuracy: 0.8854
Epoch 5/20
112/112 [==============================] - 5s 45ms/step - loss: 0.2286 - accuracy: 0.9015
Epoch 6/20
112/112 [==============================] - 5s 45ms/step - loss: 0.1901 - accuracy: 0.9175
Epoch 7/20
112/112 [==============================] - 5s 45ms/step - loss: 0.1537 - accuracy: 0.9337
Epoch 8/20
112/112 [==============================] - 5s 45ms/step - loss: 0.1624 - accuracy: 0.9292
Epoch 9/20
112/112 [==============================] - 5s 46ms/step - loss: 0.1374 - accuracy: 0.9421
Epoch 10/20
112/112 [==============================] - 5s 45ms/step - loss: 0.1304 - accuracy: 0.9420

In [15]:

# Set random seed for reproducibility
np.random.seed(0)
tf.random.set_seed(0)

# Load the EfficientNetB7 model pre-trained on ImageNet data
efficientnet_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers
for layer in efficientnet_model.layers:
    layer.trainable = False

# Build a new model using EfficientNetB7 as a feature extractor
with tf.device('/GPU:0'):
    model_efficientnet = Sequential()
    model_efficientnet.add(efficientnet_model)
    model_efficientnet.add(Flatten())
    model_efficientnet.add(Dense(256, activation='relu', kernel_initializer='glorot_normal', bias_initializer='zeros'))
    model_efficientnet.add(Dropout(0.5, seed=0))
    model_efficientnet.add(Dense(2, activation='softmax', kernel_initializer='glorot_normal', bias_initializer='zeros'))

    # Compile the model
    model_efficientnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    evaluate_model(model_efficientnet, 'EfficientNetB0', data, labels)



[INFO] Fold 1 / 10 for EfficientNetB0
Epoch 1/20
112/112 [==============================] - 22s 44ms/step - loss: 0.9221 - accuracy: 0.8009
Epoch 2/20
112/112 [==============================] - 5s 41ms/step - loss: 0.2988 - accuracy: 0.8868
Epoch 3/20
112/112 [==============================] - 5s 41ms/step - loss: 0.2455 - accuracy: 0.9043
Epoch 4/20
112/112 [==============================] - 5s 41ms/step - loss: 0.2151 - accuracy: 0.9105
Epoch 5/20
112/112 [==============================] - 5s 41ms/step - loss: 0.1646 - accuracy: 0.9323
Epoch 6/20
112/112 [==============================] - 5s 41ms/step - loss: 0.1524 - accuracy: 0.9410
Epoch 7/20
112/112 [==============================] - 5s 41ms/step - loss: 0.1351 - accuracy: 0.9418
Epoch 8/20
112/112 [==============================] - 5s 41ms/step - loss: 0.1276 - accuracy: 0.9479
Epoch 9/20
112/112 [==============================] - 5s 41ms/step - loss: 0.1208 - accuracy: 0.9491
Epoch 10/20
112/112 [==============================

In [16]:
from tensorflow.keras.applications import ResNet50

# Set random seed for reproducibility
np.random.seed(0)
tf.random.set_seed(0)

# Load the ResNet50 model pre-trained on ImageNet data
resnet50_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers
for layer in resnet50_model.layers:
    layer.trainable = False

# Build a new model using ResNet50 as a feature extractor
with tf.device('/GPU:0'):
    model_resnet50 = Sequential()
    model_resnet50.add(resnet50_model)
    model_resnet50.add(Flatten())
    model_resnet50.add(Dense(256, activation='relu', kernel_initializer='glorot_normal', bias_initializer='zeros'))
    model_resnet50.add(Dropout(0.5, seed=0))
    model_resnet50.add(Dense(2, activation='softmax', kernel_initializer='glorot_normal', bias_initializer='zeros'))

    # Compile the model
    model_resnet50.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    evaluate_model(model_resnet50, 'ResNet50', data, labels)



[INFO] Fold 1 / 10 for ResNet50
Epoch 1/20
112/112 [==============================] - 15s 54ms/step - loss: 3.4286 - accuracy: 0.7346
Epoch 2/20
112/112 [==============================] - 5s 49ms/step - loss: 0.3588 - accuracy: 0.8327
Epoch 3/20
112/112 [==============================] - 5s 48ms/step - loss: 0.2699 - accuracy: 0.8750
Epoch 4/20
112/112 [==============================] - 5s 48ms/step - loss: 0.2368 - accuracy: 0.8863
Epoch 5/20
112/112 [==============================] - 5s 48ms/step - loss: 0.2093 - accuracy: 0.9069
Epoch 6/20
112/112 [==============================] - 5s 49ms/step - loss: 0.1773 - accuracy: 0.9213
Epoch 7/20
112/112 [==============================] - 5s 49ms/step - loss: 0.1498 - accuracy: 0.9348
Epoch 8/20
112/112 [==============================] - 6s 49ms/step - loss: 0.1450 - accuracy: 0.9382
Epoch 9/20
112/112 [==============================] - 6s 50ms/step - loss: 0.1265 - accuracy: 0.9453
Epoch 10/20
112/112 [==============================] - 6s

In [17]:
# Save results to a CSV file
results_df.to_csv('DS3_baseline.csv', index=False)

In [18]:
results_df

,Model,Average Accuracy,Average Sensitivity,Average Specificity,Average AUC-ROC,Average MCC,Average Precision,Average F1 Score,Memory Used (MB),Time (s)
0,LR,0.777217,0.868299,0.577823,0.723061,0.465433,0.818345,0.842536,1044.820312,4276.968050
1,NB,0.714755,0.682630,0.785081,0.733855,0.435374,0.874373,0.766510,1022.550781,358.817622
2,KNN,0.813627,0.903475,0.616935,0.760205,0.551450,0.837795,0.869292,1036.117188,302.535838
3,Random Forest,0.843468,0.941793,0.628226,0.785010,0.622286,0.847341,0.892009,1081.343750,594.872937
4,LGBM,0.863573,0.944005,0.687500,0.815752,0.673396,0.868695,0.904738,20401.667969,10676.102058
5,VGG16,0.980529,0.988949,0.962097,0.975523,0.954180,0.983171,0.986016,891.832031,554.066261
6,EfficientNetB0,0.984956,0.991528,0.970565,0.981046,0.964571,0.987016,0.989221,168.324219,502.637363
7,ResNet50,0.984450,0.991712,0.968548,0.980130,0.963450,0.986006,0.988824,121.386719,689.414380
